In [1]:
from PIL import Image
import numpy as np
import os
import glob

# Create output directory
os.makedirs("./data/overlay", exist_ok=True)

# Get all .tiff and .tif files from images directory
image_files = glob.glob("./data/images/*.tiff") + glob.glob("./data/images/*.tif")

print(f"Found {len(image_files)} image files to process\n")

for img_path in image_files:
    # Get base filename (without path and extension)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    
    # Try to find corresponding mask with either extension
    mask_path = None
    for mask_ext in ['_masks.tif', '_masks.tiff']:
        candidate_path = f"./data/instance_masks/{base_name}{mask_ext}"
        if os.path.exists(candidate_path):
            mask_path = candidate_path
            break
    
    # Check if mask exists
    if mask_path is None:
        print(f"Warning: Mask not found for {base_name}, skipping...")
        continue
    
    print(f"Processing: {base_name}")
    
    # Load original image
    img = Image.open(img_path)
    img_np = np.array(img)

    # Load mask
    mask = Image.open(mask_path)
    mask_np = np.array(mask)

    # If mask is RGB, take one channel
    if mask_np.ndim == 3:
        mask_np = mask_np[..., 0]

    # Normalize image for display - improved version with subtle contrast enhancement
    if img.mode not in ['RGB', 'L', 'RGBA']:
        if img.mode in ['I', 'I;16', 'F']:
            # Normalize 16-bit or float images to 8-bit with subtle contrast stretching
            img_array = img_np.copy().astype(np.float32)
            
            # Use gentler percentile-based contrast stretching
            p_low = np.percentile(img_array, 0.5)
            p_high = np.percentile(img_array, 99.5)
            
            # Clip and normalize
            img_array = np.clip(img_array, p_low, p_high)
            img_array = ((img_array - p_low) / (p_high - p_low) * 255).astype(np.uint8)
            
            img = Image.fromarray(img_array, mode='L')
            img_disp = img_array
        else:
            img = img.convert('L')
            img_disp = np.array(img)
    else:
        # Apply subtle contrast stretching to 8-bit images too
        img_array = img_np.copy().astype(np.float32)
        p_low = np.percentile(img_array, 0.5)
        p_high = np.percentile(img_array, 99.5)
        img_array = np.clip(img_array, p_low, p_high)
        img_disp = ((img_array - p_low) / (p_high - p_low) * 255).astype(np.uint8)

    print(f"  Image shape: {img_np.shape}, dtype: {img_np.dtype}, mode: {img.mode}")
    print(f"  Mask shape: {mask_np.shape}, unique values: {len(np.unique(mask_np))}")

    # -----------------------
    # Pre-compute colors for ALL objects (fixed seed for consistency)
    # -----------------------
    rng = np.random.default_rng(42)  # Fixed seed
    all_instance_ids = np.unique(mask_np)
    color_map = {}
    for instance_id in all_instance_ids:
        if instance_id == 0:  # Skip background
            continue
        color_map[instance_id] = rng.random(3)

    print(f"  Created color map for {len(color_map)} unique objects")

    # -----------------------
    # Create full image overlay
    # -----------------------
    rgb_mask_full = np.zeros((*mask_np.shape, 3), dtype=np.float32)
    for instance_id in np.unique(mask_np):
        if instance_id == 0:
            continue
        rgb_mask_full[mask_np == instance_id] = color_map[instance_id]

    # Create full overlay
    overlay_full = np.stack([img_disp, img_disp, img_disp], axis=-1).astype(np.float32) / 255.0
    overlay_full = overlay_full * 0.6 + rgb_mask_full * 0.4
    overlay_full = (overlay_full * 255).astype(np.uint8)

    # Save full overlay with same filename
    output_path = f"./data/overlay/{base_name}_overlay.tif"
    Image.fromarray(overlay_full).save(output_path)
    print(f"  Saved overlay to {output_path}\n")

print("All files processed!")

Found 1 image files to process

Processing: 20211222_125057_petiole4_00012
  Image shape: (2560, 2560), dtype: float32, mode: L
  Mask shape: (2560, 2560), unique values: 2085
  Created color map for 2084 unique objects
  Saved overlay to ./data/overlay/20211222_125057_petiole4_00012_overlay.tif

All files processed!


In [2]:
from PIL import Image
import numpy as np
import os
import glob
import tifffile as tiff

def rgb_tif_to_gray_tif(
    input_tif: str,
    output_tif: str,
    method: str = "luminance"
):
    """
    Convert a 3-channel TIFF to a grayscale TIFF with shape [1, H, W].
    """
    img = tiff.imread(input_tif)

    if img.ndim != 3 or img.shape[-1] != 3:
        raise ValueError(f"Expected 3-channel TIFF, got shape {img.shape}")

    in_dtype = img.dtype
    img = img.astype(np.float32)

    if method == "luminance":
        gray = (
            0.299 * img[..., 0] +
            0.587 * img[..., 1] +
            0.114 * img[..., 2]
        )
    elif method == "average":
        gray = img.mean(axis=-1)
    else:
        raise ValueError("method must be 'luminance' or 'average'")

    # Restore dtype
    if np.issubdtype(in_dtype, np.integer):
        max_val = np.iinfo(in_dtype).max
        gray = np.clip(gray, 0, max_val).astype(in_dtype)

    # 🔴 ADD CHANNEL AXIS → (1, H, W)
    gray = gray[np.newaxis, :, :]

    # Save without squeezing
    tiff.imwrite(
        output_tif,
        gray,
        photometric="minisblack"
    )

    print(f"  Saved grayscale TIFF with shape {gray.shape}: {output_tif}")


# Create output directory
os.makedirs("./data/overlay", exist_ok=True)

# Get all .tiff and .tif files from images directory
image_files = glob.glob("./data/images/*.tiff") + glob.glob("./data/images/*.tif")

print(f"Found {len(image_files)} image files to process\n")

for img_path in image_files:
    # Get base filename (without path and extension)
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    
    # Try to find corresponding mask with either extension
    mask_path = None
    for mask_ext in ['_masks.tif', '_masks.tiff']:
        candidate_path = f"./data/instance_masks/{base_name}{mask_ext}"
        if os.path.exists(candidate_path):
            mask_path = candidate_path
            break
    
    # Check if mask exists
    if mask_path is None:
        print(f"Warning: Mask not found for {base_name}, skipping...")
        continue
    
    print(f"Processing: {base_name}")
    
    # Define output paths
    output_path = f"./data/overlay/{base_name}_overlay.tif"
    gray_output_path = f"./data/overlay/{base_name}_overlay_gray.tif"
    
    # Check if gray overlay already exists
    gray_exists = os.path.exists(gray_output_path)
    
    # Skip if gray overlay exists (no need to create RGB overlay either)
    if gray_exists:
        print(f"  Gray overlay already exists, skipping...")
        continue
    
    # Load original image
    img = Image.open(img_path)
    img_np = np.array(img)

    # Load mask
    mask = Image.open(mask_path)
    mask_np = np.array(mask)

    # If mask is RGB, take one channel
    if mask_np.ndim == 3:
        mask_np = mask_np[..., 0]

    # Normalize image for display - improved version with subtle contrast enhancement
    if img.mode not in ['RGB', 'L', 'RGBA']:
        if img.mode in ['I', 'I;16', 'F']:
            # Normalize 16-bit or float images to 8-bit with subtle contrast stretching
            img_array = img_np.copy().astype(np.float32)
            
            # Use gentler percentile-based contrast stretching
            p_low = np.percentile(img_array, 0.5)
            p_high = np.percentile(img_array, 99.5)
            
            # Clip and normalize
            img_array = np.clip(img_array, p_low, p_high)
            img_array = ((img_array - p_low) / (p_high - p_low) * 255).astype(np.uint8)
            
            img = Image.fromarray(img_array, mode='L')
            img_disp = img_array
        else:
            img = img.convert('L')
            img_disp = np.array(img)
    else:
        # Apply subtle contrast stretching to 8-bit images too
        img_array = img_np.copy().astype(np.float32)
        p_low = np.percentile(img_array, 0.5)
        p_high = np.percentile(img_array, 99.5)
        img_array = np.clip(img_array, p_low, p_high)
        img_disp = ((img_array - p_low) / (p_high - p_low) * 255).astype(np.uint8)

    print(f"  Image shape: {img_np.shape}, dtype: {img_np.dtype}, mode: {img.mode}")
    print(f"  Mask shape: {mask_np.shape}, unique values: {len(np.unique(mask_np))}")

    # -----------------------
    # Pre-compute colors for ALL objects (fixed seed for consistency)
    # -----------------------
    rng = np.random.default_rng(42)  # Fixed seed
    all_instance_ids = np.unique(mask_np)
    color_map = {}
    for instance_id in all_instance_ids:
        if instance_id == 0:  # Skip background
            continue
        color_map[instance_id] = rng.random(3)

    print(f"  Created color map for {len(color_map)} unique objects")

    # -----------------------
    # Create full image overlay
    # -----------------------
    rgb_mask_full = np.zeros((*mask_np.shape, 3), dtype=np.float32)
    for instance_id in np.unique(mask_np):
        if instance_id == 0:
            continue
        rgb_mask_full[mask_np == instance_id] = color_map[instance_id]

    # Create full overlay
    overlay_full = np.stack([img_disp, img_disp, img_disp], axis=-1).astype(np.float32) / 255.0
    overlay_full = overlay_full * 0.6 + rgb_mask_full * 0.4
    overlay_full = (overlay_full * 255).astype(np.uint8)

    # Save full RGB overlay
    Image.fromarray(overlay_full).save(output_path)
    print(f"  Saved RGB overlay to {output_path}")
    
    # Convert RGB overlay to grayscale
    rgb_tif_to_gray_tif(
        output_path,
        gray_output_path,
        method="luminance"
    )
    print()

print("All files processed!")

Found 1 image files to process

Processing: 20211222_125057_petiole4_00012
  Image shape: (2560, 2560), dtype: float32, mode: L
  Mask shape: (2560, 2560), unique values: 2085
  Created color map for 2084 unique objects
  Saved RGB overlay to ./data/overlay/20211222_125057_petiole4_00012_overlay.tif
  Saved grayscale TIFF with shape (1, 2560, 2560): ./data/overlay/20211222_125057_petiole4_00012_overlay_gray.tif

All files processed!
